In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
import sys
sys.path.append('/content/drive/My Drive/Colab Data/sudoku')

Fundamental

In [12]:
rows = 'ABCDEFGHI'
cols = '123456789'
cols_rev = cols[::-1]
is_diagonal = 1

In [13]:
def cross(a, b):
  return [s+t for s in a for t in b]

In [14]:
#boxes
boxes = cross(rows, cols)
boxes[0:10]

['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'B1']

In [15]:
#units
row_units = [cross(r, cols) for r in rows]

column_units = [cross(rows, c) for c in cols]

square_units = [cross(rs, cs) for rs in ('ABC', 'DEF', 'GHI') for cs in ('123', '456', '789')]

unitlist = row_units + column_units + square_units

d1_units = [[rows[i]+cols[i] for i in range(len(rows))]]
d2_units = [[rows[i]+cols_rev[i] for i in range(len(rows))]]

In [16]:
unitlist

[['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9'],
 ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9'],
 ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9'],
 ['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9'],
 ['E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9'],
 ['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9'],
 ['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9'],
 ['H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'H9'],
 ['I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9'],
 ['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1'],
 ['A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2', 'I2'],
 ['A3', 'B3', 'C3', 'D3', 'E3', 'F3', 'G3', 'H3', 'I3'],
 ['A4', 'B4', 'C4', 'D4', 'E4', 'F4', 'G4', 'H4', 'I4'],
 ['A5', 'B5', 'C5', 'D5', 'E5', 'F5', 'G5', 'H5', 'I5'],
 ['A6', 'B6', 'C6', 'D6', 'E6', 'F6', 'G6', 'H6', 'I6'],
 ['A7', 'B7', 'C7', 'D7', 'E7', 'F7', 'G7', 'H7', 'I7'],
 ['A8', 'B8', 'C8', 'D8', 'E8', 'F8', 'G8', 'H8', 'I8'],
 ['A9', 'B9', 'C9', 'D9', 'E9',

In [17]:
if is_diagonal == 1:
  unitlist = row_units + column_units + square_units + d1_units + d2_units
else:
  unitlist = row_units + column_units + square_units

In [18]:
#dictionary construction
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
units['A1']

[['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9'],
 ['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1'],
 ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3'],
 ['A1', 'B2', 'C3', 'D4', 'E5', 'F6', 'G7', 'H8', 'I9']]

In [19]:
peers = dict((s, set(sum(units[s], []))-set([s])) for s in boxes)
peers['A1']

{'A2',
 'A3',
 'A4',
 'A5',
 'A6',
 'A7',
 'A8',
 'A9',
 'B1',
 'B2',
 'B3',
 'C1',
 'C2',
 'C3',
 'D1',
 'D4',
 'E1',
 'E5',
 'F1',
 'F6',
 'G1',
 'G7',
 'H1',
 'H8',
 'I1',
 'I9'}

In [20]:
from utils import *

def grid_values(grid):
  chars = []
  digits = '123456789'
  for c in grid:
      if c in digits:
        chars.append(c)
      if c == '.':
        chars.append(digits)
  assert len(chars) == 81
  return dict(zip(boxes, chars))

In [33]:
def display(values):
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return

elimination

In [22]:
def eliminate(values):
  solved_values = [box for box in values.keys() if len(values[box]) == 1]
  for box in solved_values:
    digit = values[box]
    for peer in peers[box]:
      values[peer] = values[peer].replace(digit,'')
  return values

In [23]:
grid = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
grid_dict = grid_values(grid)
grid_dict_eliminated = eliminate(grid_dict)
display(grid_dict)

   4     4578    3   |  149     2     147  |   6     5789    5   
   9     2467   247  |   3      47     5   |   78     8      1   
   25    257     1   |   8      79     6   |   4    23579   2357 
---------------------+---------------------+---------------------
  1345   1345    8   |         3456    2   |   9    134567 34567 
   7    123459  249  |  1459   369     14  |   1    13456    8   
  1345  13459    6   |   7     3459    8   |   2     1345   345  
---------------------+---------------------+---------------------
  1234  12347        |   6     478     9   |   5    12478   247  
   8      16     47  |   2     457     3   |   17    467     9   
   6    24679    5   |   4      1      47  |   3    24678   2467 


only choice

In [24]:
def only_choice(values):
  for unit in unitlist:
    for digit in '123456789':
      dplaces = [box for box in unit if digit in values[box]]
      if len(dplaces) == 1:
        values[dplaces[0]] = digit
  return values

In [25]:
grid = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
grid_dict = grid_values(grid)
grid_dict_eliminated = eliminate(grid_dict)
grid_only_choice = only_choice(grid_dict_eliminated)
display(grid_only_choice)

   4      8      3   |   9      2      1   |   6     5789    5   
   9      6     247  |   3      4      5   |   8      8      1   
   25    257     1   |   8      7      6   |   4    23579   2357 
---------------------+---------------------+---------------------
  1345   1345    8   |         3456    2   |   9    134567 34567 
   7      2      9   |   5      3      1   |   1    13456    8   
  1345  13459    6   |   7     3459    8   |   2     1345   345  
---------------------+---------------------+---------------------
  1234  12347        |   6      8      9   |   5    12478   247  
   8      1      47  |   2      5      3   |   17     7      9   
   6      9      5   |   4      1      7   |   3      8      2   


constraint propagation

In [29]:
def reduce_puzzle(values):
  solved_values = [box for box in values.keys() if len(values[box]) == 1]
  stalled = False
  while not stalled:
    solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])
    values = eliminate(values)
    values = only_choice(values)
    solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
    stalled = solved_values_before == solved_values_after
    if len([box for box in values.keys() if len(values[box]) == 0]):
      return False
  return values

In [34]:
grid = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
grid_dict = grid_values(grid)
grid_dict_eliminated = eliminate(grid_dict)
grid_only_choice = only_choice(grid_dict_eliminated)
grid_reduce_puzzle = reduce_puzzle(grid_only_choice)
display(grid_reduce_puzzle)


TypeError: ignored

In [35]:
def search(values):
  values = reduce_puzzle(values)
  if values is False:
    return False
  if all(len(values[s]) == 1 for s in boxes):
    return values
  n, s = min((len(values[s]), s) for s in boxes if len(values[s]) > 1)
  for value in values[s]:
    new_sudoku = values.copy()
    new_sudoku[s] = value
    attempt = search(new_sudoku)
    if attempt:
      return attempt

In [36]:
grid = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
grid_dict = grid_values(grid)
grid_dict_eliminated = eliminate(grid_dict)
grid_only_choice = only_choice(grid_dict_eliminated)
grid_reduce_puzzle = reduce_puzzle(grid_only_choice)
grid_search = search(grid_reduce_puzzle)
display(grid_search)

AttributeError: ignored

naked twins with diagonal sudoku

In [37]:
def naked_twins(values):
  potential_twins = [box for box in values.keys() if len(values[box]) == 2]
  naked_twins = [[box1, box2] for box1 in potential_twins
                 for box2 in peers[box1]
                 if set(values[box1])==set(values[box2])]
  for i in range(len(naked_twins)):
    box1 = naked_twins[i][0]
    box2 = naked_twins[i][1]

    peers1 = set(peers[box1])
    peers2 = set(peers[box2])
    peers_int = peers1 and peers2

    for peer_val in peers_int:
      if len(values[peer_val])>2:
        for rm_val in values[box1]:
          values = assign_value(values, peer_val, values[peer_val].replace(rm_val,''))
  return values

In [38]:
grid = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
grid_dict = grid_values(grid)
grid_dict_eliminated = eliminate(grid_dict)
grid_only_choice = only_choice(grid_dict_eliminated)
grid_reduce_puzzle = reduce_puzzle(grid_only_choice)
grid_search = search(grid_reduce_puzzle)
diagonal_sudoku_grid = naked_twins(grid_search)
display(diagonal_sudoku_grid)

2 6 7 |9 4 5 |3 8 1 
8 5 3 |7 1 6 |2 4 9 
4 9 1 |8 2 3 |5 7 6 
------+------+------
5 7 6 |4 3 8 |1 9 2 
3 8 4 |1 9 2 |6 5 7 
1 2 9 |6 5 7 |4 3 8 
------+------+------
6 4 2 |3 7 9 |8 1 5 
9 3 5 |2 8 1 |7 6 4 
7 1 8 |5 6 4 |9 2 3 


In [39]:
def solve(grid):
  grid_dict = grid_values(grid)
  grid_dict_eliminated = eliminate(grid_dict)
  grid_only_choice = only_choice(grid_dict_eliminated)
  grid_reduce_puzzle = reduce_puzzle(grid_only_choice)
  grid_search = search(grid_reduce_puzzle)
  diag_sudoku_grid = naked_twins(grid_search)

  return diag_sudoku_grid

In [40]:
if __name__ == '__main__':
  diag_sudoku_grid = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
  display(solve(diag_sudoku_grid))

2 6 7 |9 4 5 |3 8 1 
8 5 3 |7 1 6 |2 4 9 
4 9 1 |8 2 3 |5 7 6 
------+------+------
5 7 6 |4 3 8 |1 9 2 
3 8 4 |1 9 2 |6 5 7 
1 2 9 |6 5 7 |4 3 8 
------+------+------
6 4 2 |3 7 9 |8 1 5 
9 3 5 |2 8 1 |7 6 4 
7 1 8 |5 6 4 |9 2 3 
